# Evaluating Groups of Restaurants

Picking up on the noticed correlation of number of ratings and rating, we want to further examine possible natures of this connection. As mentioned in the previous notebook, an omitted variable, that is correlated with the 'nr_ratings' parameter and affects the 'rating' parameter, might bias the regression.

One could think that the type of a restaurant must be decisive over many outcomes. Some restaurants for instance are trying to serve the highest possible amount of costumers, while others focus on quality and ambient. Sushi tends to be more expensive than burgers. Italian restaurants are frequented during dinner time while Fast Food peaks at lunch.

We want to further examine this phenomenon and therefore try to cluster the restaurants in our data:

## Overview

In order to form a group, senseful categories are needed. The original csv file contains a column 'angebot' which specifies the type of food a restaurant is offering. Lets load the dataframe from previous notebook and append the 'angebot' information:

In [ ]:
import distance
from fuzzywuzzy import fuzz
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import re, seaborn as sns
from sklearn import cluster
from sklearn import linear_model

# Load dataframe from previous notebook
data = pd.read_pickle(r'data/modif_restaurants.pkl')
modif_df = pd.DataFrame(data)

# Load original csv file
data2 = pd.read_csv(r'data/restaurants.csv',  delimiter=';')
restaurants_df = pd.DataFrame(data2, columns=[
    'unique_id',
    'angebot',])

# Merge dataframes
restaurants_df.set_index('unique_id', inplace=True)  # prepare for merging
df = pd.concat([modif_df, restaurants_df], axis=1, join='inner')

Now we can examine, what the most rated restaurants are offering that makes them so popular:

In [ ]:
# Create new dataframe with restaurants rated more than 1000 times
often_rated_restaurants = df.drop(df[df.nr_ratings < 1000].index, inplace=False)  # filter dataframe
often_rated_restaurants.sort_values(by=['nr_ratings'], inplace=True, ascending=False)  # sort
count_series = often_rated_restaurants.name.str.count("McDonald")  # count in dataframe

display(often_rated_restaurants[['name', 'nr_ratings', 'price_lvl', 'angebot']])
print(often_rated_restaurants['price_lvl'].value_counts())
print("\n")
print("Number of McDonald's Franchises: " + str(count_series.sum()))

The majority of restaurants with more than 1000 reviews are located in the "Moderate" price level. While this is coherent with the total sample average, the distribution of the conditional dataframe still deviates from the expected distribution. Especially, places without price category are underepresented. Also worth mentioning is, that there is only one "Expensive" restaurant in the selection. We already discussed possible reasons for both phenomenons in previous notebook.

If we take a look at each restaurants name and 'angebot', it stands out that multiple franchise restaurants (e.g. 6 times McDonald's) are represented in the selection. This will be the first approach for creating groups (see **Franchise vs Privately Owned**).

One can also identify a variety of food types offered. While Burger, Pizza and Pasta seem to thrive, traditional German food is represented as well. We will have a closer look at this distribution within the second clustering approach (see **Type of Food**).

## 1) Franchise vs Privately Owned

Some franchises in the selected dataframe are recognized quickly. We can create a list of those. Additionally, all restaurants that occur more than one time in the dataframe shall be appended to the list: 

In [ ]:
# Create empty list that will hold unique ids of all franchise restaurants
franchise_ids = []

# Create list with known franchises
known_franchises = ["McDonald", "Subway", "Osteria", "Bonanza Coffee", "Dominos", "Hard Rock Cafe", "BLOCK HOUSE",
                    "Jim Block", "Dolores"]

# Detect multiple entries in dataframe
name_distribution = df['name'].value_counts()
for name, count in name_distribution.iteritems():
    if count > 1:
        known_franchises.append(name)  # append name to list
    else:
        pass

# Iterate over list wih known franchises
for index, restaurant in df.iterrows():
    for franchise in known_franchises:
        if franchise in restaurant['name']:
            franchise_ids.append(index)  # append unique id as matching criteria to list
        else:
            pass

# Remove duplicates
franchise_ids = list(set(franchise_ids))

print("Number of Franchise Restaurants: " + str(len(franchise_ids)))

Since we gained new information about the restaurants we can expand the dataframe:

In [ ]:
# Append Franchise category as column to dataframe
df.insert(loc=7, column='franchise', value=int)

# Fill column with booleans
for index, restaurant in df.iterrows():
    if index in franchise_ids:
        df.at[index, 'franchise'] = 1
    else:
        df.at[index, 'franchise'] = 0

# Convert type to bool
df['franchise'] = df['franchise'].astype('bool')

The information is stored as a boolean variable - 1 if Franchise, 0 if not. Are there any deviations within the groups from the sample average?

In [ ]:
# Create dataframe with group means
means = df.groupby('franchise')[['rating', 'nr_ratings']].mean().round(decimals=2)
means = means.rename(columns={'rating': "mean_rating", 'nr_ratings': "mean_nr_ratings"})

# Create dataframe with group standard deviations
stds = df.groupby('franchise')[['rating', 'nr_ratings']].std().round(decimals=2)
stds = stds.rename(columns={'rating': "std_rating", 'nr_ratings': "std_nr_ratings"})

# Merge dataframes
oview = pd.concat([means, stds], axis=1)

# Add mean and standard deviation of total sample as row to output dataframeata
describe_df = df.describe()
mean_rating = describe_df.loc['mean', 'rating']
mean_nr_ratings = describe_df.loc['mean', 'nr_ratings']
std_rating = describe_df.loc['std', 'rating']
std_nr_ratings = describe_df.loc['std', 'nr_ratings']

# Count frequency of each row
counter_total = describe_df.loc['count', 'rating']
counter_series = df['franchise'].value_counts()

# Append series as column to output dataframe
oview['freq'] = counter_series
oview.loc['Total'] = [mean_rating, mean_nr_ratings, std_rating, std_nr_ratings, counter_total]

# Format output dataframe
oview = oview.astype({'freq': int})
oview = oview.round(decimals=2)

print(oview)

Indeed, franchises tend to have more ratings with on average worse ratings than restaurants that are privately owned. Keep in mind that there is likely a bias, since we partly selected the franchises based on the conditional dataframe ('nr_ratings' > 1000). Lets know examine, if the negative correlation of 'nr_ratings' on 'rating' discussed in previous notebook holds:

**Multivariate Linear Regression**

In [ ]:
# Create new dataframe exclusiv outlier
small_restaurants_df = df.drop(df[df.nr_ratings > 5000].index, inplace=False)

# Multivariate linear regression of number of ratings and franchise on rating
X = small_restaurants_df[['nr_ratings', 'franchise']]
y = small_restaurants_df['rating']
regr = linear_model.LinearRegression()
regr.fit(X, y)

print("Intercept: \n", regr.intercept_)
print("Coefficients: ")
for coef in regr.coef_:
    print(format(coef, 'f'))

The coefficients still indicate a negative correlation of 'nr_ratings' and 'rating'. However, when controlling for the restaurant type the amount almost halved. This happens because the number of ratings is correlated to the type of restaurant (i.e. 'franchise') as well. The latter affects a restaurants rating by 0.3545 stars on average.

**3D Plot**

In [ ]:
%matplotlib notebook

# Create 3d scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Variables
X = small_restaurants_df['nr_ratings']
Y = small_restaurants_df['franchise']
z = small_restaurants_df['rating']

# Plot points
ax.scatter(X, Y, z)
cmap = ListedColormap(sns.color_palette("husl", 2).as_hex())  # get colors
sc = ax.scatter(X, Y, z, s=40, c=Y, marker='o', cmap=cmap, alpha=1)

# Labeling
ax.set_xlabel('nr_ratings')
ax.set_ylabel('franchise')
ax.set_zlabel('rating')

# Produce a legend
L = plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=1, title="franchise")
L.get_texts()[0].set_text('False')
L.get_texts()[1].set_text('True')   
              
# Rotate the axes
for angle in range(0, 360):
    ax.view_init(30, angle)
    plt.draw()
    plt.pause(.001)
    
plt.show()

## 2) Type of Food

There is no common approach for grouping restaurants. Obviously assigning every food type to their origin region would be convenient. Yet there are a lot of crossover and nationality-independent restaurants especially in a multicultural city like Berlin. We therefore need to think of more diverse categories - or let an algorithm do just that for us:

### Manually

Based on the knowledge of the restaurant 'angebot' data, one can create sensful categories to match the restaurants with afterwards. The following code is mainly based on string comparision using the Levenshtein distance and substring characteristics:

In [ ]:
# Create list with known food types
food_types = ["französisch", "asiatisch", "deutsch", "italienisch", "russisch", "griechisch", "thai", "mexikanisch", "indisch", "japanisch", "chinesisch", "koreanisch", "international", "arabisch", "orientalisch",
              "suppe", "burger", "döner", "sushi", "kaffee", "kuchen", "steak", "pizza", "pasta", "salat", "bbq", "bowl",
              "regional", "vegan", "vegetarisch", "saisonal"]

# Append new column to dataframe
df.insert(loc=8, column='food_type', value=None)

# Check for food type in dataframe
for index, restaurant in df.iterrows():
    f_types = []
    angebot = restaurant['angebot']
    angebot_list = angebot.split()
    for word in angebot_list:
        word = word.lower()
        for type in food_types:
            levenshtein_distance = fuzz.ratio(type, word)
            if type in word and type not in f_types:
                f_types.append(type)
            elif levenshtein_distance > 85 and type not in f_types:
               f_types.append(type)
            else:
                pass
    df.at[index, 'food_type'] = f_types
    
display(df[['name', 'angebot', 'food_type']])

The Levenshtein distance and the substring function perform quite good at identifying the prior specified food types within the 'angebot' column. There are problems though, when the offer is described in any other language than German. Hence, the following output should be evaluated critically:

In [ ]:
# Set new index to match array
new_indexed_df = df.reset_index()

# Create array with matching index
col_array = new_indexed_df["food_type"].to_numpy(na_value=None)

# Create and prepare dataframe to display output in
eval_df = pd.DataFrame({'food_type': food_types,
                        'mean_rating': None,
                        'mean_nr_ratings': None,
                        'std_rating': None,
                        'std_nr_ratings': None,
                        'frequency': None,
                        })

eval_df.set_index('food_type', inplace=True)  # set index

# Creating mask for calculating conditional benchmarks
for food_type in food_types:  # loop through prior specified list
    mask = []  # set up list
    for restaurant in col_array:
        cond = food_type in restaurant
        mask.append(cond)  # fill list with True/False
    mask_array = np.array(mask) # list to array
    food_type_df = new_indexed_df[mask_array]  # fancy indexing with mask array
    describe_df = food_type_df.describe()  # calculate output
    mean_rating = describe_df.loc['mean', 'rating']
    mean_nr_ratings = describe_df.loc['mean', 'nr_ratings']
    std_rating = describe_df.loc['std', 'rating']
    std_nr_ratings = describe_df.loc['std', 'nr_ratings']
    freq = describe_df.loc['count', 'rating']
    eval_df.at[food_type, 'mean_rating'] = mean_rating
    eval_df.at[food_type, 'mean_nr_ratings'] = mean_nr_ratings
    eval_df.at[food_type, 'std_rating'] = std_rating
    eval_df.at[food_type, 'std_nr_ratings'] = std_nr_ratings
    eval_df.at[food_type, 'frequency'] = freq

# Sort by frequency descending
eval_df.sort_values(by=['frequency'], axis=0, ascending=False, inplace=True)
eval_df['frequency'] = eval_df['frequency'].astype(int)
print(eval_df)

There are some remarkable differences among the restaurant types. If we take for example Italian food which is in general rather expensive and compare it to restaurants offering Burgers, we see that the latter receive on average almost three times as many ratings. This might be due to higher customer quantity. The average Italian restaurant however, scores better ratings (~0.2 stars). 

The datframe with both the franchise and food type information shall be used for the geographical visualization with kepler later on:

In [ ]:
df.to_csv(r'data/geo_restaurants.csv', sep=',', encoding='utf-8', index=True)
df.to_pickle(r'data/geo_restaurants.pkl')

**Clustering Algorithm - DBSCAN**

What makes DBSCAN a convinient choice for clustering the food offers in the Restaurant dataset, is its non-parametric property. The number of clusters does not need to be specified in advance. Therefore, the clustering result only depends on epsilon (i.e. radius of the neighborhood) and the minPts (i.e. minimum number of points within neighborhood to be classified as core point). In addition, the algorithm detects outliers as noise points.

The description within the 'angebot' column of each restaurant is split into words of which each one is appended to a list. Since the entries  vary a lot - some places are described in sentences others in bullet points - we first need to remove meaningless fillwords. Fortunately, lists with fillwords exist for most languages. In addition, punctuation marks and other meaningless characters need to be striped:

In [ ]:
# Create list with words in 'angebot' column
angebot_words = []
for index, restaurant in df.iterrows():  # iterate over dataframe rows
    list_words = restaurant['angebot'].split()  # split string into words
    for word in list_words:
        angebot_words.append(word.lower())  # no capitalization wanted
print("Length raw list: " + str(len(angebot_words)))

# Load list with German fillwords - source: https://gist.github.com/makelefy/f5ff045c08f8982d130ccd4c5b616019
fill_words = []
with open ("data/stopWords.txt") as file_object:
    for line in file_object:
        fillword = line.replace("\n", "")
        fill_words.append(fillword)

# Create new list without fillwords
raw_food_words = [word for word in angebot_words if word not in fill_words]  # append if not fillword
food_words = []
for raw_word in raw_food_words:
    clean_word = re.sub('[^a-zäüöß]+', '', raw_word)  # clean strip
    food_words.append(clean_word)

print("Length cleansed list: " + str(len(food_words)))

<br>Grupping unlabeled data requires information about the similarity of the instances. Since we aim on clustering strings, common distances like the euclidean norm cannot be used. One of the most appropriate functions for string comparison is the Levensthein distance. It measures the difference between two strings by counting the number of necessary character edits to convert one string into the other.

There are over 3500 instances in our list. A square matrix with the distance between each point and in both ways has over 12 million entries. Calculating the matrix took my system (AMD Ryzen 7 4700U, 16GB RAM, Winx64) about 15 minutes. You will find the corresponding code in the cell below. The size of the  csv file is around 300MB and cannot be uploaded to GitHub (100MB limit). If you do not want to run the cell, then I suggest you download the file [here](https://www.dropbox.com/sh/p8aq23l99jwezw1/AAChdIf7zpL0eiyVFlyXeEyta?dl=0). Make sure that you move the file to the 'data' folder afterwards.

In [ ]:
# Calculate Levensthein distance metrix !!!long runtime!!!
words = np.asarray(food_words)  # list to array
lev_metric = np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])  # create distance matrix
np.savetxt("data/lev_metric.csv", lev_metric, delimiter=",")

The Levensthein distance matrix can now be used to run the DBSCAN algortihm:

In [ ]:
# Clustering
lev_metric = np.loadtxt('data/lev_metric.csv', delimiter=",")
clustering = cluster.dbscan(lev_metric, metric='precomputed', eps=3, min_samples=5)  # run DBSCAN
cluster_labels = clustering[1]  # output is tupel of 2 arrays
food_words_array = np.array(food_words)  # convert list into array
cluster_count=np.max(cluster_labels)+1  # count number of clusters
for cluster_number in range(cluster_count):
    cluster_x = food_words_array[cluster_labels == cluster_number]  # boolean masking and fancy indexing
    print("\n")
    print("Cluster " + str(cluster_number) + " :")
    print(cluster_x)

The clustering of round about 3500 instances is done within a few seconds. If you look at the output, you will see that each cluster mainly contains duplicates or words with a little different spelling. Noise points are located in the first cluster. In this application DBSCAN basically counts the frequency of similiar words within the 'angebot' column and groups them in a cluster if their count exceeds the 'min_samples' value. Note that the number and the type of clusters depend heavenly on the specified parameters (i.e. 'eps' & 'min_sample'). I suggest to try tweaking the paramaters yourself in order to see their impact.

We specified that there need to be at least four ('min_sample') other instances within a Levensthein distance of three ('eps') for a point to be classified as a core point. When there is a point in reach of a core point but with less than three neighboors, the cluster edge is formed. So, are the clusters created by DBSCAN making sense? We can check this by comparing all restaurants of one exemplary cluster:

In [ ]:
# Choose cluster from DBSCAN output with eps=3 and min_samples=5
cluster40 = ['thailandische', 'holländische', 'thailändische', 'thailändische','thailändisch', 'thailändische',
             'thailändische', 'thailändische', 'thailändisch', 'thailändisch', 'thailändische']

# Create list of indexes from restaurants in cluster
restaurants_in_cluster40 = []
for idx, restaurant in df.iterrows():
    for word in cluster40:
        if word in restaurant['angebot'] and idx not in restaurants_in_cluster40:
            restaurants_in_cluster40.append(idx)

# Select dataframe columns from list
cluster40_df = df.loc[restaurants_in_cluster40]  # indexing with list
pd.set_option('display.max_colwidth', None)
display(cluster40_df[["name", "delivery", "franchise", "price_lvl", "nr_ratings", "rating", "food_type", "angebot"]])

Looking at the 'angebot' column of the dataframe, we see that 4 out of 5 restaurants are actually serving Thai food. Those were already recognized by our manual clustering efforts. The one outlier is serving dutch fries which translate to 'holländische Fritten'. The Levensthein distance  between the latter and 'thailändische' is 3. Therefore, it is considered a core point and thus part of the cluster.

It turns out that the DBSCAN algorithm performs quite good at detecting similiar strings. Unfortunately, it is not able to group the  food types in sensful categories. Some categories (e.g. 'produkte') have none or little meaning. Furthermore, some categories that humans might group together (e.g. 'burger' and 'american') cannot be recognized by DBSCAN. Since the results of the manual clustering seem to be more appropriate, we won't append the algorithm results to the Restaurants dataframe.